In [76]:
import pandas as pd
import numpy as np
import folium
import folium.plugins as plugins
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import time
import os
import re
import fileinput
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
from folium.plugins import Search
import matplotlib

In [77]:
fp = "SHAPES/municipios_simp.json"
cidadesBR = pd.read_csv("csv/muniBR-ALL.csv",delimiter=',')
dataSet = pd.read_csv("csv/OB_DATA_2016.csv",sep=",",low_memory=False)
dataFrameBase = pd.DataFrame(dataSet, columns = ['CODMUNRES','IDADE','CAUSABAS'])


In [78]:
onlyAsma = dataFrameBase[dataFrameBase['CAUSABAS'].str.contains('J45')]#doencas respiratorias

In [ ]:
onlyAsma

In [79]:
otherDisease = dataFrameBase[dataFrameBase['CAUSABAS'].str.contains('J')]#doencas respiratorias

In [ ]:
otherDisease

In [80]:
data = gpd.read_file(fp)

In [ ]:
data

In [81]:
data['OB_ASMA']=0
data['OB_RESP']=0
data['POPULACAO']=10000 #debug
data['OB_ASMA_POP']=0
data['OB_RESP_POP']=0

In [83]:
numCities = len(cidadesBR)
for i in range(0,numCities):
	codMun=cidadesBR.iloc[i, 1]
	popMun=cidadesBR.iloc[i, 3]
	data.loc[data.GEOCODIGO.str.contains(str(codMun)), 'POPULACAO'] =popMun

values = onlyAsma['CODMUNRES'].value_counts().keys().tolist()
counts = onlyAsma['CODMUNRES'].value_counts().tolist()
numCounts = len(counts)

In [84]:
mapa = folium.Map(location=[-15.788497,-47.879873],tiles='Mapbox Bright', zoom_start=4)

In [ ]:
for i in range(0,numCounts):
	data.OB_ASMA.loc[data.GEOCODIGO.str.contains(str(values[i]))]=counts[i]
	#folium.Marker(location=[-15.788497,-47.879873], popup=Popup(counts[i])).add_to(mapa)
data.loc['OB_ASMA_POP']=round(100000*data['OB_ASMA']/data['POPULACAO'])

In [86]:
valuesD = otherDisease['CODMUNRES'].value_counts().keys().tolist()
countsD = otherDisease['CODMUNRES'].value_counts().tolist()
numCountsD = len(countsD)

In [88]:
for i in range(0,numCountsD):
    data.OB_RESP.loc[data.GEOCODIGO.str.contains(str(valuesD[i]))]=countsD[i]
    #folium.Marker(location=[-15.788497,-47.879873], popup=Popup(counts[i])).add_to(mapa)
data.loc['OB_RESP_POP']=round(100000*data['OB_RESP']/data['POPULACAO'])

/home/marciel/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [137]:
data['OB_RESP_POP']=round(100000*data['OB_RESP']/data['POPULACAO'],1)
data['OB_ASMA_POP']=round(100000*data['OB_ASMA']/data['POPULACAO'],1)

In [138]:
round(data['OB_RESP_POP'],1)

0        87.1
1        56.4
2        23.9
3        58.5
4        68.4
5         4.9
6       106.6
7        85.9
8       122.8
9       107.5
10       82.4
11       83.5
12       85.3
13       52.9
14       40.3
15       35.0
16       53.3
17       12.8
18      100.1
19       81.7
20       53.7
21       44.0
22      121.7
23      125.7
24       46.7
25       42.7
26       68.4
27       86.9
28      109.8
29       33.8
        ...  
5540    153.8
5541    121.3
5542     97.4
5543    150.3
5544     33.0
5545    204.6
5546    115.4
5547     97.7
5548    101.1
5549     81.2
5550     36.6
5551    103.3
5552     82.4
5553    146.2
5554    130.8
5555    115.8
5556    170.4
5557     34.2
5558    192.7
5559    136.5
5560     45.5
5561    332.9
5562    129.4
5563      0.0
5564    140.6
5565     98.8
5566    137.4
5567    177.7
5568    143.2
5569     50.7
Name: OB_RESP_POP, Length: 5570, dtype: float64

In [136]:
round(data['OB_ASMA_POP'],1)

0        0.0
1        0.0
2        0.0
3        0.0
4       11.0
5        1.0
6        0.0
7        0.0
8        5.0
9        0.0
10       0.0
11       0.0
12       0.0
13       0.0
14       0.0
15       0.0
16       0.0
17       0.0
18       9.0
19       0.0
20       0.0
21       0.0
22       0.0
23       1.0
24       0.0
25       0.0
26       0.0
27       0.0
28       0.0
29       1.0
        ... 
5540    17.0
5541     0.0
5542    10.0
5543     0.0
5544     0.0
5545     0.0
5546     0.0
5547     3.0
5548     0.0
5549     0.0
5550     0.0
5551     0.0
5552     0.0
5553     0.0
5554     0.0
5555     0.0
5556     0.0
5557     0.0
5558    16.0
5559     1.0
5560     0.0
5561     0.0
5562     0.0
5563     0.0
5564     0.0
5565     0.0
5566     1.0
5567     0.0
5568     0.0
5569     0.0
Name: OB_ASMA_POP, Length: 5570, dtype: float64

In [191]:
fmap= folium.Map(location=[-20,-40], 
                    zoom_start=5,
                    tiles='cartodbpositron',
                    control_scale=True)


In [170]:

data['RGBA'] = 0
data['RGBA_RESP'] = 0

In [189]:
ColorMapOB = plt.get_cmap('OrRd',12)
rgba_asma =  ColorMapOB((data['OB_ASMA_POP'] ** 2 / (1 + data['OB_ASMA_POP'] ** 2)))

In [190]:
ColorMapOB = plt.get_cmap('OrRd',12)
rgba_resp =  ColorMapOB((data['OB_RESP_POP'] ** 2 / (1 + data['OB_RESP_POP'] ** 2)))

(0.143343, 0.522773, 0.556295, 1.0)

In [155]:
rgba_asma

array([[0.2298057 , 0.29871797, 0.75368315, 1.        ],
       [0.2298057 , 0.29871797, 0.75368315, 1.        ],
       [0.2298057 , 0.29871797, 0.75368315, 1.        ],
       ...,
       [0.2298057 , 0.29871797, 0.75368315, 1.        ],
       [0.2298057 , 0.29871797, 0.75368315, 1.        ],
       [0.2298057 , 0.29871797, 0.75368315, 1.        ]])

In [192]:
data['RGBA'] = rgba_asma
data['RGBA_RESP'] = rgba_resp

In [162]:
data.loc[0,'RGBA']

0.2298057

In [193]:
#rgb2hex(Blues(data['RGBA'][1]))
#rgb2hex(Blues(data['RGBA']))
#for i in range(0,numCities):
for i in range(0,numCities):
    data.loc[i,'RGBA'] = matplotlib.colors.to_hex(rgba_asma[i])
    data.loc[i,'RGBA_RESP'] = matplotlib.colors.to_hex(rgba_resp[i])

In [194]:
asmaGroup = folium.FeatureGroup(name='Taxa de Obitos por asma')
asmaMap = folium.GeoJson(
        data[['NOME', 'geometry','OB_ASMA_POP','RGBA']].to_json(), 
        name='Taxa de Obitos por Asma/ 100 mil hab',
        show=True,
        style_function=lambda feature: {
            'fillColor': feature['properties']['RGBA'],
            'color': 'white',
            'weight': 0,
            'fillOpacity':0.5
        },   
        highlight_function=lambda x: {'weight':1, 
                                      'color':'black',
                                      'fillOpacity':0.9},    
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NOME','OB_ASMA_POP'],
            aliases=['Cidade:','Obitos:'],                              
        ),  
).add_to(asmaGroup)
asmaGroup.add_to(fmap)

In [195]:
respGroup = folium.FeatureGroup(name='Taxa de Obitos por todas as Doenças Respiratórias')
respMap = folium.GeoJson(
        data[['NOME', 'geometry','OB_RESP_POP','RGBA_RESP']].to_json(), 
        name='Taxa de Obitos por outras doencas respiratorias/ 100 mil hab',
        show=False,
        style_function=lambda feature: {
            'fillColor': feature['properties']['RGBA_RESP'],
            'color': 'white',
            'weight': 0,
            'fillOpacity':0.5
        },   
        highlight_function=lambda x: {'weight':1, 
                                      'color':'black',
                                      'fillOpacity':0.9},    
        tooltip=folium.features.GeoJsonTooltip(
            fields=['NOME','OB_RESP_POP'],
            aliases=['Cidade:','Obitos:'],                              
        ),  
).add_to(respGroup)
respGroup.add_to(fmap)

In [ ]:
fmap

In [196]:

citysearch = Search(
    layer=asmaMap,
    geom_type='Polygon',
    placeholder='Busca Cidade',
    collapsed=True,
    search_label='NOME'
).add_to(fmap)


In [197]:
folium.LayerControl(collapsed=True).add_to(fmap)
fmap.save("mapaT.html") 

In [ ]:
# todo https://python-graph-gallery.com/313-bubble-map-with-folium/

In [ ]:
# todo https://python-graph-gallery.com/25-histogram-with-several-variables-seaborn/

In [ ]:
# todo https://python-graph-gallery.com/391-radar-chart-with-several-individuals/
# exibir o percentual de obitos por asma por região, IC de 95%

In [ ]:
# todo https://python-graph-gallery.com/13-percent-stacked-barplot/
# exibir o percentual de obitos por asma por faixa etaria

In [ ]:
# todo https://python-graph-gallery.com/171-basic-venn-diagram-with-3-groups/